# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
display(data.head(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Надо привести данные к такому виду, чтобы можно было анализировать( без пропусков, дубликатов и т.п.)
Пропуски в стаже и доходе.Значения в столбце "стаж" отрицательные или слишком "большие" отрицательные, значения в столбце образование записаны в разном регистре.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data['days_employed'] = data['days_employed'].apply(abs) # берет значения по модулю
data.loc[data['days_employed'] >30000, 'days_employed'] = data.loc[data['days_employed'] >30000, 'days_employed'] / 24
# привели значения стажа в положительные значения


#print('Пропуски до:', data['days_employed'].isna().sum())
data.loc[data['days_employed'].isna(), 'days_employed'] = data['days_employed'].median() #заполняем пропуски стажа медианным значением
#print('Пропуски после:', data['days_employed'].isna().sum())


data.loc[data['total_income'].isna(), 'total_income'] = data['total_income'].median()#заполняем пропуски дохода медианным значением
#print('Пропуски после:', data['total_income'].isna().sum())





#print(data['children'].unique())
data.loc[data['children'] == -1, 'children'] = 0 # предполагая , что -1 ,это отсустсвие детей, а 20 это 2(ошибка), меняем значения в столбце дети
data.loc[data['children'] == 20, 'children'] = 2
    

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Т.к. значения стажа были отрицательными, использовали apply(abs) для перевода их в положительные . Также пропущенные значения стажа и дохода заменили медианными значениями. В столбце 'children' значение -1, заменили на 0, предполагая, что это отсутствие детей, а значение 20 , подозревая , что это ошибка, поменяли на более правдоподобное 2.

### Замена типа данных

In [3]:
data['days_employed'] = data['days_employed'].astype(int)#меняем вещественные значения столбца стаж на целые ,
#с доходом ,я считаю, это не нужно
display(data.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Вещественные значения столбца стажа меняем на целые. Менять значения столбца доход на целое , считаю нецелесообразным.

### Обработка дубликатов

In [4]:
data['education'] = data['education'].str.lower()#меняем регистр в столбце образование
#print(data['education'].unique())

data = data.drop_duplicates().reset_index(drop = True)#удаляем полностью дублирующиеся строки
#print(data.duplicated().sum())

**Вывод**

В столбце "образование" приводим значения к нижнему регистру.
Дублирующиеся строки удаляем полностью.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

text = data['purpose'].unique() #выделяем леммы в значениях столбца с целями получения кредита
razdelitel = ' '
text = razdelitel.join(text)
lemmas = m.lemmatize(text)
#print(Counter(lemmas))


def purpose_lemmas(row): 
    purpose = row['purpose']
    lemmas_purpose = m.lemmatize(purpose)
    if ('недвижимость' in lemmas_purpose or 'жилье' in lemmas_purpose ):
        return 'недвижимость'
    elif ('автомобиль' in lemmas_purpose):
        return 'автомобиль'
    elif ('образование' in lemmas_purpose):
        return 'образование'
    elif ('свадьба' in lemmas_purpose):
        return 'свадьба'
    else:
        return 'цель не определена'
data['lemmas_purpose'] = data.apply(purpose_lemmas, axis = 1)




display(data.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость


**Вывод**

Лемматизируем цели кредита по наиболее часто встречающимся словам и выводим в отдельный столбец.

### Категоризация данных

In [6]:
#print(data['total_income'].min())# ищем минимальный заработок

#print(data['total_income'].max())# ищем максимальный заработок

def income_group(income):# создаю функцию для категоризации клиентов по уровню заработной платы
    if income < 50000:
        return 'низкий  доход'
    elif income < 100000:
        return 'средний  доход'
    elif income < 300000:
        return 'высокий  доход'
    else:
        return 'очень высокий доход'
    
data['income_group'] = data['total_income'].apply(income_group)# добавляем новый столбец по категориям
display(data.head(50))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,высокий доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,высокий доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,высокий доход
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,высокий доход
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость,высокий доход
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость,высокий доход
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование,высокий доход
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба,средний доход
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость,высокий доход


**Вывод**

Числа в столбце доход немного шокируют(не очень похоже на правду), но будем работать отвлеченно от реальности. Разделим заемщиков на 4 группы : низкий доход, средний, высокий, очень высокий. Категории запишем в отдельный столбец.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
ch_off = data[data['children'] == 0]['children'].count()  #изучаем количество заемщиков без детей
print('Количество заемщиков без детей:', ch_off)

 
ch_off_debt_off = data[data['children'] == 0][data[data['children'] == 0]['debt'] == 0]['debt'].count() #считаем количество бездетных заемщиков без задолженностей

ch_on = data[data['children'] !=0]['children'].count() #изучаем количество заемщиков с детьми
print('Количество заемщиков с детьми:', ch_on)

ch_on_debt_off = data[data['children'] != 0][data[data['children'] != 0]['debt'] == 0]['debt'].count()#считаем количество заемщиков с детьми без задолженностей

print('Заемщики без детей возвращают кредит в срок в ',(ch_off_debt_off / ch_off * 100).round(2), '% случаев')
print('Заемщики с детьми возвращают кредит в срок в ',(ch_on_debt_off / ch_on * 100).round(2), '% случаев')


Количество заемщиков без детей: 14138
Количество заемщиков с детьми: 7316
Заемщики без детей возвращают кредит в срок в  92.47 % случаев
Заемщики с детьми возвращают кредит в срок в  90.75 % случаев


**Вывод**

Т.к. вопрос стоит таким образом , чтобы найти зависимость между наличием детей в принципе, или отсутствием их у заемщика , с возвратом кредита в срок, поэтому проанализировал данные таким образом. И в итоге вывод, что заемщики без детей возвращают деньги вовремя примерно на 1,75% лучше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:

df_family_status_id = data.pivot_table(index = ['family_status_id'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# формируем сводную таблицу по 'family_status_id' ,где в строке данные по кол-ву задолженностей , общему количеству , проценту задолженностей

df_family_status_id.columns = ['debt', 'total', '%'] 
df_family_status_id = df_family_status_id.sort_values(by = ['total'], ascending = False) #сортируем наш датафрейм по убыванию
df_family_status_id['%'] = df_family_status_id['%'].round(3)*100

df_family_status_id

,debt,total,%
family_status_id,,,
0,931,12339,7.5
1,388,4151,9.3
4,274,2810,9.8
3,85,1195,7.1
2,63,959,6.6


**Вывод**

Как ни странно, но хуже всего с возвратом кредита в срок дело обстоит у не женатых и состоящих в гражданском браке 9,8% и 9,3% соответственно. Лучше всего отдают деньги вовремя овдовевшие и разведенные.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
df_income_group = data.pivot_table(index = ['income_group'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# формируем сводную таблицу по 'income_group' ,где в строке данные по кол-ву задолженностей , общему количеству , проценту задолженностей

df_income_group.columns = ['debt', 'total', '%'] 
df_income_group = df_income_group.sort_values(by = ['total'], ascending = False) #сортируем наш датафрейм по убыванию
df_income_group['%'] = df_income_group['%'].round(3)*100

df_income_group

,debt,total,%
income_group,,,
высокий доход,1281,15509,8.3
средний доход,331,4091,8.1
очень высокий доход,106,1482,7.2
низкий доход,23,372,6.2


**Вывод**

Лучше всего с возвратом денег в срок у людей с низким доходом , а хуже всего у людей с высоким и средним доходом 8,3% и 8,1% соответственно.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
df_lemmas_purpose = data.pivot_table(index = ['lemmas_purpose'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
# формируем сводную таблицу по 'lemmas_purpose' ,где в строке данные по кол-ву задолженностей , общему количеству , проценту задолженностей

df_lemmas_purpose.columns = ['debt', 'total', '%'] 
df_lemmas_purpose = df_lemmas_purpose.sort_values(by = ['total'], ascending = False) #сортируем наш датафрейм по убыванию
df_lemmas_purpose['%'] = df_lemmas_purpose['%'].round(3)*100

df_lemmas_purpose

,debt,total,%
lemmas_purpose,,,
недвижимость,782,10811,7.2
автомобиль,403,4306,9.4
образование,370,4013,9.2
свадьба,186,2324,8.0


**Вывод**

Кредит на недвижимость оказался наименее рискованым , просрочки 7,2%. А с кредитом на автомобиль или образование ситуация похуже, просрочки 9,4% и 9,2% соответственно.

## Шаг 4. Общий вывод

  Решая поставленые задачи и отвечая на заданые вопросы, в ходе моего проекта , я получил следующие выводы:
1. Наиболее востребованый кредит - на недвижимость(10811).
2. Чаще всего за кредитом обращаются люди с высоким доходом(в нашем датафрейме > 100000р.)(15509).
3. Кредит хотят взять в большинстве своем женатые люди(12339).
4. у заемщиков чаще всего нет детей(14138).
  Обобщая полученые результаты в заданиях рискну предположить наиболее и наименее перспективного заемщика:
Наиболее перспективный: бездетный вдовец с низким доходом, желающий взять кредит на недвижимость.
Наименее перспективный: неженатый человек с детьми, высоким уровнем дохода и целью купить автомобиль.

Выводы сделаны на основе предоставленных данных и поставленных задач(не является руководством к действиям).


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.